# Feature Engineering

In [1]:
import pandas as pd
import numpy as np
import os
from scipy.stats.mstats import winsorize
from datetime import datetime

In [2]:
# Пути к данным
RAW_PATH = 'data/raw'
PROCESSED_PATH = 'data/processed'
os.makedirs(PROCESSED_PATH, exist_ok=True)

In [3]:
# Загрузка данных
orders = pd.read_csv(os.path.join(RAW_PATH, 'olist_orders_dataset.csv'),
                     parse_dates=['order_purchase_timestamp'])
order_items = pd.read_csv(
    os.path.join(RAW_PATH, 'olist_order_items_dataset.csv'))
customers = pd.read_csv(os.path.join(RAW_PATH, 'olist_customers_dataset.csv'))
payments = pd.read_csv(
    os.path.join(RAW_PATH, 'olist_order_payments_dataset.csv'))
products = pd.read_csv(os.path.join(RAW_PATH, 'olist_products_dataset.csv'))

In [4]:
# Объединение таблиц
df = orders.merge(order_items, on='order_id', how='left')
df = df.merge(customers[['customer_id', 'customer_unique_id']],
              on='customer_id',
              how='left')
df = df.merge(payments, on='order_id', how='left')
df = df.merge(products[['product_id', 'product_category_name']],
              on='product_id',
              how='left')

In [5]:
# Time-split: первые 2 года — исторические признаки, остаток — future LTV
min_date = df['order_purchase_timestamp'].min()
cutoff_date = min_date + pd.DateOffset(years=2)

historical = df[df['order_purchase_timestamp'] < cutoff_date]
future = df[df['order_purchase_timestamp'] >= cutoff_date]

# Target = future LTV
future_ltv = future.groupby(
    'customer_unique_id')['payment_value'].sum().reset_index()
future_ltv.columns = ['customer_unique_id', 'future_ltv']

In [6]:
# RFM + доп. фичи на исторических данных
rfm = historical.groupby('customer_unique_id').agg({
    'order_purchase_timestamp': [
        lambda x: (cutoff_date - x.max()).days,  # recency
        'nunique',  # frequency
        lambda x: x.sort_values().diff().dt.days.mean()
        if len(x) > 1 else 0,  # avg_days_between_orders
        lambda x: (x.min() - min_date).days  # first_purchase_day
    ],
    'payment_value':
    'sum',  # monetary
    'product_category_name':
    'nunique'  # category_diversity
}).reset_index()

rfm.columns = [
    'customer_unique_id', 'recency', 'frequency', 'avg_days_between_orders',
    'first_purchase_day', 'monetary', 'category_diversity'
]

In [7]:
# Обработка выбросов: winsorize monetary на 99% квантиль
rfm['monetary'] = winsorize(rfm['monetary'], limits=[0, 0.01])

In [8]:
# One-hot для payment_type 
payment_main = historical.groupby(['customer_unique_id', 'payment_type'
                                   ]).size().reset_index(name='count')
payment_main = payment_main.loc[payment_main.groupby('customer_unique_id')
                                ['count'].idxmax()]

payment_ohe = pd.get_dummies(payment_main['payment_type'],
                             prefix='payment',
                             dtype=int)

# merge без суффиксов
rfm = rfm.merge(payment_main[['customer_unique_id']],
                left_on='customer_unique_id',
                right_on='customer_unique_id',
                how='left')
rfm = rfm.merge(payment_ohe, left_index=True, right_index=True,
                how='left').fillna(0)

In [9]:
# Churn: если recency > 90 дней — считаем churn
rfm['churn'] = (rfm['recency'] > 90).astype(int)

In [10]:
# Добавляем future LTV
rfm = rfm.merge(future_ltv, on='customer_unique_id', how='left')
rfm['future_ltv'] = rfm['future_ltv'].fillna(0)

In [13]:
# Финальные признаки (без recency и churn — чтобы избежать leakage)
final_df = rfm.drop(columns=['recency', 'churn'])

# Проверка типов данных
print("Типы данных перед сохранением:")
print(final_df.dtypes)


Типы данных перед сохранением:
customer_unique_id          object
frequency                    int64
avg_days_between_orders    float64
first_purchase_day           int64
monetary                   float64
category_diversity           int64
payment_boleto             float64
payment_credit_card        float64
payment_debit_card         float64
payment_not_defined        float64
payment_voucher            float64
future_ltv                 float64
dtype: object


In [14]:

# Сохраняем
final_df.to_csv(os.path.join(PROCESSED_PATH, 'features.csv'), index=False)
print("Features saved to 'data/processed/features.csv'")

Features saved to 'data/processed/features.csv'


In [15]:
final_df

,customer_unique_id,frequency,avg_days_between_orders,first_purchase_day,monetary,category_diversity,payment_boleto,payment_credit_card,payment_debit_card,payment_not_defined,payment_voucher,future_ltv
0,0000366f3b9a7992bf8c76cfdf3221e2,1,0.0,612,141.90,1,0.0,1.0,0.0,0.0,0.0,0.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,0.0,609,27.19,1,0.0,1.0,0.0,0.0,0.0,0.0
2,0000f46a3911fa3c0805444483337064,1,0.0,186,86.22,1,0.0,1.0,0.0,0.0,0.0,0.0
3,0000f6ccb0745a6a4b88665a16c9f078,1,0.0,402,43.62,1,0.0,1.0,0.0,0.0,0.0,0.0
4,0004aac84e0df4da2b147fca70cf8255,1,0.0,435,196.89,1,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
96086,fffcf5a5ff07b0908bd4e2dbc735a684,1,0.0,276,1687.59,1,0.0,0.0,0.0,0.0,0.0,0.0
96087,fffea47cd6d3cc0a88bd621562a9d061,1,0.0,461,84.58,1,0.0,1.0,0.0,0.0,0.0,0.0
96088,ffff371b4d645b6ecea244b27531430a,1,0.0,155,112.46,1,1.0,0.0,0.0,0.0,0.0,0.0
96089,ffff5962728ec6157033ef9805bacc48,1,0.0,604,133.69,1,0.0,1.0,0.0,0.0,0.0,0.0
